# Day 5 - Supply Stacks
## Data

In [20]:
class Animal():
    def __init__(self, name, sound):
        self.name = name
        self.sound = sound
    
    def speak(self):
        print(f"{self.name}: {self.sound}")


class Cat(Animal):
    def __init__(self, name):
        super().__init__(name, 'Meeeeeowwwww')

        
class Dog(Animal):
    def __init__(self, name):
        super().__init__(name, 'Wooof!')

        
animals = [
    Cat('Oji'),
    Dog('Keli'),
    Cat('Skit Skat'),
    Dog('Tumba')
]

for animal in animals:
    animal.speak()


Oji: Meeeeeowwwww
Keli: Wooof!
Skit Skat: Meeeeeowwwww
Tumba: Wooof!


['f', 'o', 'o']

In [44]:
class Stack(object):
    def __init__(self, id, crates):
        self.id = id
        self.crates = list(crates)

    def __repr__(self):
        crates =  "".join(f"[{crate}]" for crate in self.crates)
        return f"{self.id}: {crates}"
    
    def remove(self):
        """Removes the top crate on the stack and returns what it was."""
        return self.crates.pop()
    
    def add(self, crate):
        """Puts the crate at the top of the stack."""
        self.crates.append(crate)
    
    
class Step(object):
    def __init__(self, quantity, from_stack, to_stack):
        self.quantity = quantity
        self.from_stack = from_stack
        self.to_stack = to_stack
    
    def __repr__(self):
        return f"move {self.quantity} from {self.from_stack} to {self.to_stack}"
    
    def execute(self):
        for i in range(self.quantity):
            removed_crate = self.from_stack.remove()
            self.to_stack.add(removed_crate)
            print(f"Moved [{removed_crate}] from {self.from_stack.id} to {self.to_stack.id}")
            
    
class Procedure(object):
    def __init__(self, stacks, steps):
        self.stacks = stacks
        self.steps = steps
    
    def __repr__(self):
        stacks = "\n".join(str(stack) for stack in self.stacks)
        steps = "\n".join(str(step) for step in self.steps)
        
        return f"{stacks}\n\n{steps}\n\n"
    
    def execute(self):
        while self.steps:
            self.steps[0].execute()
            print("------------------------")
            self.steps.pop(0)
            print(self)
        

example_input = """
    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 
 
move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2
"""
        
example_stacks = [
    Stack(1, 'ZN'),
    Stack(2, 'MCD'),
    Stack(3, 'P')
]

example_steps = [
    Step(1, example_stacks[1], example_stacks[0]),
    Step(3, example_stacks[0], example_stacks[2]),
    Step(2, example_stacks[1], example_stacks[0]),
    Step(1, example_stacks[0], example_stacks[1])
]

example_procedure = Procedure(example_stacks, example_steps)

print(example_procedure)
example_procedure.execute()

1: [Z][N]
2: [M][C][D]
3: [P]

move 1 from 2: [M][C][D] to 1: [Z][N]
move 3 from 1: [Z][N] to 3: [P]
move 2 from 2: [M][C][D] to 1: [Z][N]
move 1 from 1: [Z][N] to 2: [M][C][D]


Moved [D] from 2 to 1
------------------------
1: [Z][N][D]
2: [M][C]
3: [P]

move 3 from 1: [Z][N][D] to 3: [P]
move 2 from 2: [M][C] to 1: [Z][N][D]
move 1 from 1: [Z][N][D] to 2: [M][C]


Moved [D] from 1 to 3
Moved [N] from 1 to 3
Moved [Z] from 1 to 3
------------------------
1: 
2: [M][C]
3: [P][D][N][Z]

move 2 from 2: [M][C] to 1: 
move 1 from 1:  to 2: [M][C]


Moved [C] from 2 to 1
Moved [M] from 2 to 1
------------------------
1: [C][M]
2: 
3: [P][D][N][Z]

move 1 from 1: [C][M] to 2: 


Moved [M] from 1 to 2
------------------------
1: [C]
2: [M]
3: [P][D][N][Z]






In [3]:
from aocd import get_data

raw_data = get_data(year=2022, day=5)
print(raw_data)



            [M] [S] [S]            
        [M] [N] [L] [T] [Q]        
[G]     [P] [C] [F] [G] [T]        
[B]     [J] [D] [P] [V] [F] [F]    
[D]     [D] [G] [C] [Z] [H] [B] [G]
[C] [G] [Q] [L] [N] [D] [M] [D] [Q]
[P] [V] [S] [S] [B] [B] [Z] [M] [C]
[R] [H] [N] [P] [J] [Q] [B] [C] [F]
 1   2   3   4   5   6   7   8   9 

move 1 from 7 to 4
move 3 from 4 to 7
move 4 from 3 to 4
move 5 from 6 to 9
move 1 from 8 to 1
move 2 from 3 to 2
move 3 from 4 to 6
move 1 from 3 to 6
move 9 from 7 to 1
move 1 from 2 to 4
move 3 from 4 to 9
move 4 from 9 to 8
move 6 from 8 to 2
move 1 from 8 to 6
move 1 from 4 to 1
move 11 from 1 to 7
move 1 from 4 to 7
move 7 from 2 to 5
move 5 from 6 to 3
move 2 from 4 to 3
move 2 from 5 to 9
move 1 from 8 to 6
move 3 from 1 to 5
move 2 from 6 to 9
move 1 from 4 to 8
move 2 from 2 to 1
move 7 from 5 to 9
move 6 from 3 to 6
move 1 from 2 to 5
move 1 from 3 to 8
move 12 from 7 to 3
move 1 from 1 to 8
move 2 from 1 to 9
move 20 from 9 to 5
move 1 from 1 to 7
move 5 

## Part 1

In [65]:
def is_contained(container, containee):
    """Returns True iff the container contains the containee."""
    return container[0] <= containee[0] and container[1] >= containee[1]

def is_contained_pair(first_assignment, second_assignment):
    """Returns True iff one assignment in the pair fully contains the other."""
    return is_contained(first_assignment, second_assignment) or is_contained(second_assignment, first_assignment)

assert is_contained_pair((2, 4), (6, 8)) == False
assert is_contained_pair((2, 8), (3, 7)) == True
assert is_contained_pair((6, 6), (4, 6)) == True

def count_contained_pairs(assignment_pairs):
    return sum(is_contained_pair(pair[0], pair[1]) for pair in assignment_pairs)
    
assert count_contained_pairs(example_assignment_pairs) == 2
count_contained_pairs(real_assignment_pairs)

464

## Part 2

In [71]:
def is_overlapping_pair(first_assignment, second_assignment):
    if first_assignment[0] >= second_assignment[0] and first_assignment[0] <= second_assignment[1]:
        return True
    elif first_assignment[1] >= second_assignment[0] and first_assignment[1] <= second_assignment[1]:
        return True
    elif is_contained_pair(first_assignment, second_assignment):
        return True
    else:
        return False

assert is_overlapping_pair((2, 4), (6, 8)) == False
assert is_overlapping_pair((2, 8), (3, 7)) == True
assert is_overlapping_pair((5, 7), (7, 9)) == True

def count_overlapping_pairs(assignment_pairs):
    return sum(is_overlapping_pair(pair[0], pair[1]) for pair in assignment_pairs)

assert count_overlapping_pairs(example_assignment_pairs) == 4
count_overlapping_pairs(real_assignment_pairs)

770